In [1]:
from tqdm import tqdm, trange
import time
#import nbimporter
import numpy as np
import math
import pandas as pd
#pd.set_option('max_columns',1000)
import os
#import my_save_load_v2
os.chdir(r'/public/hanl/jupyter_dir/database/AMUCdb/sepsis_episode/septic_shock_onset')
#filename = r'pickle_mimic_iv_derived_table_v3.1.pkl'
#my_save_load_v2.my_load(filename, globals())
#import missingno as msno 


## 输出高清图像
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
## 图像显示中文的问题
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['axes.unicode_minus']=False
#import seaborn as sns 
#sns.set(font= "Kaiti",style="ticks",font_scale=1.4)

In [2]:
# 请跳转到最后导入数据

In [3]:
# firs
def splite_data(sepsis3_tem,step=100):
    data = [sepsis3_tem.iloc[i:i+step,:] for i in range(0,sepsis3_tem.shape[0],step) ]
    return data


## 获取 vaso 前3h 输液量，未去除输液只有1分钟的数据
def fluid_3h_pre_vaso(sub_tem_sepsis3, tem_vasoactive_vasopressor,tem_fluids_bolus):
    sub_tem_vaso = pd.merge(sub_tem_sepsis3, tem_vasoactive_vasopressor, on='admissionid', how='inner')
    sub_tem_vaso = sub_tem_vaso.loc[sub_tem_vaso.vaso_starttime >= sub_tem_vaso.sofa_time, :]
    sub_tem_vaso.loc[:, 'vaso_3h_pre'] = sub_tem_vaso.vaso_starttime - 3*60 # mins   #dt.timedelta(hours=3)
    sub_tem_vaso_fluid = pd.merge(sub_tem_vaso, tem_fluids_bolus, on=['admissionid'], how='inner')

        #去除输液只有1分钟的数据
    sub_tem_vaso_fluid = sub_tem_vaso_fluid.loc[(sub_tem_vaso_fluid.loc[:, 'fluid_endtime'] - sub_tem_vaso_fluid.loc[:, 'fluid_starttime']) > 1,:]

    tem = (sub_tem_vaso_fluid.loc[:, ['vaso_starttime', 'fluid_endtime']].min(axis=1) - sub_tem_vaso_fluid.loc[:, ['vaso_3h_pre','fluid_starttime']].max(axis=1))
    tem = tem / (sub_tem_vaso_fluid.loc[:, 'fluid_endtime'] - sub_tem_vaso_fluid.loc[:, 'fluid_starttime'])

    tem_fluids_bolus_3h = tem.clip(0, tem) * sub_tem_vaso_fluid.loc[:, 'fluid']
    sub_tem_vaso_fluid.loc[:, 'fluids_bolus_3h'] = tem_fluids_bolus_3h
    sub_tem_vaso_fluid.drop(['fluid_starttime', 'fluid_endtime', 'fluid'], axis=1, inplace=True)
    sub_tem_vaso_fluid = sub_tem_vaso_fluid.groupby(
            ['admissionid', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre'], as_index=False).agg(sum)
    return sub_tem_vaso_fluid


# 获取 vaso 前 第一个 mbp值
def fluid_3h_pre_vaso_mbp(sub_tem_vaso_fluid,tem_mbp):
    # 获取 vaso 后 第一个 mbp值
    sub_tem_vaso_fluid_mbp = pd.merge(sub_tem_vaso_fluid,tem_mbp,on=['admissionid'], how='inner')

    #sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.loc[sub_tem_vaso_fluid_mbp.mbp_charttime >= sub_tem_vaso_fluid_mbp.vaso_starttime,:]
    sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.loc[ sub_tem_vaso_fluid_mbp.mbp_charttime <= sub_tem_vaso_fluid_mbp.vaso_starttime, :]

    #sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.sort_values(by=['stay_id', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre', 'mbp_charttime'])   #时间 按小到大 排序
    sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.sort_values(by=['admissionid', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre', 'mbp_charttime'],
                                                                ascending=[True,True,True,True,True,False])  # 时间 大到小 排序

    sub_tem_vaso_fluid_mbp=sub_tem_vaso_fluid_mbp.groupby(['admissionid', 'sofa_time', 'vaso_starttime', 'vaso_endtime',
           'vaso_3h_pre', 'fluids_bolus_3h'], as_index=False ).apply(lambda x:x.iloc[0,:])
    return(sub_tem_vaso_fluid_mbp)


# 获取 vaso 后 第一个 lactate 值
def fluid_3h_pre_vaso_mbp_lactate(sub_tem_vaso_fluid_mbp,tem_lactate):
    # 获取 vaso 后 第一个 lactate 值
    sub_tem_vaso_fluid_mbp_lactate = pd.merge(sub_tem_vaso_fluid_mbp, tem_lactate, on=['admissionid'], how='inner')
    sub_tem_vaso_fluid_mbp_lactate = sub_tem_vaso_fluid_mbp_lactate.loc[
                                     sub_tem_vaso_fluid_mbp_lactate.lactate_charttime >= sub_tem_vaso_fluid_mbp_lactate.vaso_starttime,
                                     :]
    sub_tem_vaso_fluid_mbp_lactate = sub_tem_vaso_fluid_mbp_lactate.sort_values(
        by=['admissionid', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre',
            'mbp_charttime', 'lactate_charttime'])
    sub_tem_vaso_fluid_mbp_lactate = sub_tem_vaso_fluid_mbp_lactate.groupby(
        ['admissionid', 'sofa_time', 'vaso_starttime', 'vaso_endtime',
         'vaso_3h_pre', 'fluids_bolus_3h', 'mbp_charttime', 'mbp'], as_index=False).apply(lambda x: x.iloc[0, :])
    return(sub_tem_vaso_fluid_mbp_lactate)


def main_code(i,num,t0,tem_sepsis3_split,tem_vasoactive_vasopressor,tem_fluids_bolus,tem_mbp,tem_lactate):
    sub_tem_sepsis3=tem_sepsis3_split[i]
    sub_tem_vaso_fluid = fluid_3h_pre_vaso(sub_tem_sepsis3,tem_vasoactive_vasopressor,tem_fluids_bolus)
    sub_tem_vaso_fluid = sub_tem_vaso_fluid.loc[sub_tem_vaso_fluid.fluids_bolus_3h >= 1000]

    sub_tem_vaso_fluid_mbp = fluid_3h_pre_vaso_mbp(sub_tem_vaso_fluid,tem_mbp)
    #sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.loc[sub_tem_vaso_fluid_mbp.mbp >= 65,:]
    sub_tem_vaso_fluid_mbp = sub_tem_vaso_fluid_mbp.loc[sub_tem_vaso_fluid_mbp.mbp <= 65,:]

    sub_tem_vaso_fluid_mbp_lactate = fluid_3h_pre_vaso_mbp_lactate(sub_tem_vaso_fluid_mbp,tem_lactate)
    sub_tem_vaso_fluid_mbp_lactate = sub_tem_vaso_fluid_mbp_lactate.loc[sub_tem_vaso_fluid_mbp_lactate.lactate >= 2,:]

    shock = sub_tem_vaso_fluid_mbp_lactate.sort_values(by=['admissionid', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_3h_pre'])
    shock = shock.groupby(['admissionid', 'sofa_time'], as_index=False).apply(lambda x:x.iloc[0,:])

    print(f'complete {i} / {num} ; spend time : { time.time() - t0 :.4f} s' )

    return(shock)


In [4]:
def splite_data(sepsis3_tem, step=100):
    data = [sepsis3_tem.iloc[i:i+step,:] for i in range(0,sepsis3_tem.shape[0],step) ]
    return data



## 获取 vaso 前3h 输液量，
def fluid_3h_pre_vaso(sub_tem_sepsis3,tem_vasoactive_vasopressor,tem_fluids_bolus,nh_bef,nh_aft):
    
    sub_tem_vaso = pd.merge(sub_tem_sepsis3, tem_vasoactive_vasopressor, on='stay_id', how='inner')
    sub_tem_vaso = sub_tem_vaso.loc[sub_tem_vaso.vaso_starttime >= sub_tem_vaso.sofa_time, :]
    sub_tem_vaso.loc[:, 'vaso_nh_pre'] = sub_tem_vaso.vaso_starttime - nh_bef * 60 ## dt.timedelta(hours=nh_bef)
    sub_tem_vaso.loc[:, 'vaso_nh_aft'] = sub_tem_vaso.vaso_starttime + nh_aft * 60 ##dt.timedelta(hours=nh_aft)

    sub_tem_vaso_fluid = pd.merge(sub_tem_vaso, tem_fluids_bolus, on=['stay_id'], how='inner')

    #去除输液只有1分钟的数据
    sub_tem_vaso_fluid = sub_tem_vaso_fluid.loc[(sub_tem_vaso_fluid.loc[:, 'fluid_endtime'] - sub_tem_vaso_fluid.loc[:, 'fluid_starttime']) > 1 ,:] ##dt.timedelta(minutes=1),:]

    tem = (sub_tem_vaso_fluid.loc[:, ['vaso_starttime', 'fluid_endtime']].min(axis=1) - sub_tem_vaso_fluid.loc[:, ['vaso_nh_pre','fluid_starttime']].max(axis=1))
    tem = tem / (sub_tem_vaso_fluid.loc[:, 'fluid_endtime'] - sub_tem_vaso_fluid.loc[:, 'fluid_starttime'])

    tem_fluids_bolus_3h = tem.clip(0, tem) * sub_tem_vaso_fluid.loc[:, 'colloid_bolus']
    sub_tem_vaso_fluid.loc[:, 'fluids_bolus_3h'] = tem_fluids_bolus_3h
    sub_tem_vaso_fluid.drop(['fluid_starttime', 'fluid_endtime', 'colloid_bolus'], axis=1, inplace=True)
    sub_tem_vaso_fluid = sub_tem_vaso_fluid.groupby(
        ['stay_id', 'sofa_time', 'vaso_starttime', 'vaso_endtime', 'vaso_nh_pre','vaso_nh_aft'], as_index=False).agg(sum)
    return sub_tem_vaso_fluid


# 获取 vaso 后 第一个 lactate 值
def lactate_aft_vaso(tem_vaso_fluid,tem_lactate):

    sub_tem_fluid_vaso_lactate = pd.merge(tem_vaso_fluid, tem_lactate, on=['stay_id'], how='inner')
    
    sub_tem_fluid_vaso_lactate = sub_tem_fluid_vaso_lactate.loc[
                                     sub_tem_fluid_vaso_lactate.lactate_charttime >= sub_tem_fluid_vaso_lactate.vaso_starttime,
                                     :]

    sub_tem_fluid_vaso_lactate = sub_tem_fluid_vaso_lactate.sort_values(
        by=['stay_id', 'sofa_time', 'vaso_starttime', 'lactate_charttime'])
    
    sub_tem_fluid_vaso_lactate = sub_tem_fluid_vaso_lactate.groupby(
        ['stay_id', 'sofa_time', 'vaso_starttime', 'fluids_bolus_3h'], 
        as_index=False).apply(lambda x: x.iloc[0, :])
    
    return(sub_tem_fluid_vaso_lactate)


## 获取 mbp( mbp<=65 的时间) 前3h 输液量
def fluid_3h_pre_mbp(sub_tem_sepsis3,tem_mbp,tem_fluids_bolus,nh_bef,nh_aft):
    
    sub_tem_mbp = pd.merge(sub_tem_sepsis3, tem_mbp, on='stay_id', how='inner')
    sub_tem_mbp = sub_tem_mbp.loc[sub_tem_mbp.mbp_charttime >= sub_tem_mbp.sofa_time, :]
    
    sub_tem_mbp.loc[:, 'mbp_nh_pre'] = sub_tem_mbp.mbp_charttime - nh_bef * 60 ##dt.timedelta(hours=nh_bef)
    sub_tem_mbp.loc[:, 'mbp_nh_aft'] = sub_tem_mbp.mbp_charttime + nh_aft * 60 ## dt.timedelta(hours=nh_aft) 
    
    sub_tem_mbp_fluid = pd.merge(sub_tem_mbp, tem_fluids_bolus, on=['stay_id'], how='inner')
    tem = (sub_tem_mbp_fluid.loc[:, ['mbp_charttime', 'fluid_endtime']].min(axis=1) - sub_tem_mbp_fluid.loc[:, ['mbp_nh_pre','fluid_starttime']].max(axis=1))
    tem = tem / (sub_tem_mbp_fluid.loc[:, 'fluid_endtime'] - sub_tem_mbp_fluid.loc[:, 'fluid_starttime'])

    tem_fluids_bolus_3h = tem.clip(0, tem) * sub_tem_mbp_fluid.loc[:, 'colloid_bolus']
    sub_tem_mbp_fluid.loc[:, 'fluids_bolus_3h'] = tem_fluids_bolus_3h
    sub_tem_mbp_fluid.drop(['fluid_starttime', 'fluid_endtime', 'colloid_bolus'], axis=1, inplace=True)
    sub_tem_mbp_fluid = sub_tem_mbp_fluid.groupby(
        ['stay_id', 'sofa_time', 'mbp_charttime','mbp','mbp_nh_pre','mbp_nh_aft'], as_index=False).agg(sum)

    return sub_tem_mbp_fluid


# 获取 mbp 后 第一个 lactate 值
def lactate_aft_mbp(sub_tem_mbp_fluid,tem_lactate):
    # 获取 mbp 后 第一个 lactate 值
    sub_tem_fluid_mbp_lactate = pd.merge(sub_tem_mbp_fluid, tem_lactate, on=['stay_id'], how='inner')
    
    sub_tem_fluid_mbp_lactate = sub_tem_fluid_mbp_lactate.loc[
                                     sub_tem_fluid_mbp_lactate.lactate_charttime >= sub_tem_fluid_mbp_lactate.mbp_charttime,
                                     :]

    sub_tem_fluid_mbp_lactate = sub_tem_fluid_mbp_lactate.sort_values(
        by=['stay_id', 'sofa_time', 'mbp_charttime', 'mbp_nh_pre','mbp_nh_aft', 'lactate_charttime'])
    sub_tem_fluid_mbp_lactate = sub_tem_fluid_mbp_lactate.groupby(
        ['stay_id', 'sofa_time', 'mbp_charttime', 'mbp',
         'mbp_nh_pre', 'mbp_nh_aft', 'fluids_bolus_3h'], as_index=False).apply(lambda x: x.iloc[0, :])
    
    return(sub_tem_fluid_mbp_lactate)



def main_shock(nh_bef,nh_aft,bolus,lac,tem_sepsis3_split,tem_fluids_bolus,tem_mbp,tem_vasopressor,tem_lactate):
    
    num = tem_sepsis3_split.__len__()
    septic_shock = pd.DataFrame()
    
    for i in trange(num):
        sub_tem_sepsis3=tem_sepsis3_split[i]
        
        # vaso 前3H 的液体复苏量
        sub_tem_vaso_fluid = fluid_3h_pre_vaso(sub_tem_sepsis3,tem_vasopressor,tem_fluids_bolus,nh_bef,nh_aft)
        sub_tem_vaso_fluid = sub_tem_vaso_fluid.loc[sub_tem_vaso_fluid.fluids_bolus_3h >= bolus]

        # vaso 之后第一个lac 值
        sub_tem_vaso_fluid_lac = lactate_aft_vaso(sub_tem_vaso_fluid,tem_lactate)
        sub_tem_vaso_fluid_lac = sub_tem_vaso_fluid_lac.loc[sub_tem_vaso_fluid_lac.lactate >= lac,:]
        sub_tem_vaso_fluid_lac = sub_tem_vaso_fluid_lac.query('vaso_starttime <= lactate_charttime <= vaso_nh_aft')

        # mbp 前3H 的液体复苏量
        sub_tem_mbp_fluid = fluid_3h_pre_mbp(sub_tem_sepsis3,tem_mbp,tem_fluids_bolus,nh_bef,nh_aft)
        sub_tem_mbp_fluid = sub_tem_mbp_fluid.loc[sub_tem_mbp_fluid.fluids_bolus_3h >= bolus]

        # mbp 之后第一个lac 值
        sub_tem_mbp_fluid_lac = lactate_aft_mbp(sub_tem_mbp_fluid,tem_lactate)
        sub_tem_mbp_fluid_lac = sub_tem_mbp_fluid_lac.loc[sub_tem_mbp_fluid_lac.lactate >= lac,:]
        sub_tem_mbp_fluid_lac = sub_tem_mbp_fluid_lac.query('mbp_charttime <= lactate_charttime <= mbp_nh_aft')

        # concate vaso and mbp
        sub = pd.concat([sub_tem_vaso_fluid_lac,sub_tem_mbp_fluid_lac])
 
        # shock  # sort by lac time
        shock = sub.sort_values(by=['stay_id', 'sofa_time', 'lactate_charttime'])
        
        shock = shock.groupby(['stay_id', 'sofa_time'], as_index=False).apply(lambda x:x.iloc[0,:])
        
        septic_shock = pd.concat([septic_shock,shock],axis=0)
        
    septic_shock.rename(columns={'vaso_nh_pre': f'vaso_{nh_bef}h_bef',
                                 'vaso_nh_aft': f'vaso_{nh_aft}h_aft',
                                 'mbp_nh_pre' : f'mbp_{nh_bef}h_bef',
                                 'mbp_nh_aft' : f'mbp_{nh_aft}h_aft',
                                },inplace=True)
    
    return septic_shock
  

## Four table ##
- fluidbanlance.csv
- lactate.csv
- mbp.csv
- vasopressors_inotropes.csv

In [5]:
os.chdir(r'/public/hanl/jupyter_dir/database/AMUCdb/get_var_from_AUMC/sql_septic_shock_onset/')

In [6]:
#VASO
df_vaso = pd.read_csv("vasopressors_inotropes.csv")
df_vaso_cut = df_vaso.loc[:,['admissionid','start_time','stop_time']]
df_vaso_cut.columns = ['stay_id','vaso_starttime','vaso_endtime']
df_vaso_cut.head(2)

,stay_id,vaso_starttime,vaso_endtime
0,0,1602,2480
1,1,50,351


In [7]:
## mbp
df_mbp = pd.read_csv("mbp.csv")
df_mbp_cut = df_mbp.loc[:,['admissionid','time','value']]
df_mbp_cut.columns = ['stay_id','mbp_charttime','mbp']
df_mbp_cut.head(2)

,stay_id,mbp_charttime,mbp
0,8704,16,89
1,8704,59,89


In [8]:
## lactate
df_lac = pd.read_csv("lactate.csv")
df_lac_cut = df_lac.loc[:,['admissionid','time','lactate']]
df_lac_cut.columns = ['stay_id','lactate_charttime','lactate']
df_lac_cut.head(2)

,stay_id,lactate_charttime,lactate
0,4,6,1.7
1,4,103,3.0


In [9]:
## fluidbanlance.csv
df_fluid = pd.read_csv("fluidbanlance.csv")
df_fluid_cut = df_fluid.loc[:,['admissionid','starttime','endtime','fluid']]
df_fluid_cut.columns = ['stay_id','fluid_starttime','fluid_endtime', 'colloid_bolus']
df_fluid_cut.head(2)


,stay_id,fluid_starttime,fluid_endtime,colloid_bolus
0,0,332,392,270.0
1,0,340,400,270.0


In [10]:
os.chdir(r'/public/hanl/jupyter_dir/database/AMUCdb/sepsis_episode/sepsis_episode/')
sepsis3 = pd.read_csv("sepsis_onset.csv")
sepsis3_cut = sepsis3.loc[:,['admissionid', 'sepsis_onset']]
sepsis3_cut.columns = ['stay_id', 'sofa_time']
sepsis3_cut.head(2)

,stay_id,sofa_time
0,2,0.0
1,3,4.0


In [11]:
tem_sepsis3 = sepsis3_cut
tem_vasopressor = df_vaso_cut
tem_fluids_bolus = df_fluid_cut
tem_lactate = df_lac_cut
tem_mbp = df_mbp_cut.query('mbp<=65') 

In [12]:
nh_bef = 3 # mbp_nh_bef
nh_aft = 6 # mbp_nh_aft
bolus = 1000 # ml
lac = 2 # 

# 分批次计算
tem_sepsis3_split = splite_data(tem_sepsis3,step=300)

septic_shock = main_shock(nh_bef,nh_aft,bolus,lac,tem_sepsis3_split,tem_fluids_bolus,tem_mbp,tem_vasopressor,tem_lactate)
   
septic_shock['onset_time'] = septic_shock.lactate_charttime
septic_shock

100%|██████████| 29/29 [02:37<00:00,  5.42s/it]


,stay_id,sofa_time,vaso_starttime,vaso_endtime,vaso_3h_bef,vaso_6h_aft,fluids_bolus_3h,lactate_charttime,lactate,mbp_charttime,mbp,mbp_3h_bef,mbp_6h_aft,onset_time
0,5.0,0.0,NaN,NaN,NaN,NaN,1157.993672,415.0,2.2,174.0,65.0,-6.0,534.0,415.0
1,11.0,0.0,124.0,199.0,-56.0,484.0,1056.647260,312.0,6.1,NaN,NaN,NaN,NaN,312.0
2,15.0,1.0,320.0,465.0,140.0,680.0,1441.894432,480.0,2.6,NaN,NaN,NaN,NaN,480.0
3,21.0,7.0,633.0,1489.0,453.0,993.0,1863.033471,649.0,2.0,NaN,NaN,NaN,NaN,649.0
4,38.0,0.0,NaN,NaN,NaN,NaN,1787.896614,131.0,6.2,123.0,64.0,-57.0,483.0,131.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,23412.0,1.0,183.0,391.0,3.0,543.0,1164.511555,196.0,12.6,NaN,NaN,NaN,NaN,196.0
19,23468.0,1.0,374.0,674.0,194.0,734.0,1965.004335,464.0,2.1,NaN,NaN,NaN,NaN,464.0
20,23480.0,3.0,874.0,965.0,694.0,1234.0,1037.424395,1181.0,10.6,NaN,NaN,NaN,NaN,1181.0
21,23488.0,0.0,703.0,756.0,523.0,1063.0,1225.227766,816.0,11.4,NaN,NaN,NaN,NaN,816.0


In [13]:
septic_shock.shape

(1353, 14)

In [14]:
os.chdir(r'/public/hanl/jupyter_dir/database/AMUCdb/sepsis_episode/septic_shock_onset/')
septic_shock_lst = septic_shock.stay_id.unique().tolist()
septic_shock.to_csv("septic_shock_onset.csv", index=False)

## 与之前的结果对比

In [20]:
df_past_shock = pd.read_csv("./v1_sepsis_definedby_day/septic_shock_onset.csv")
past_shock_lst = df_past_shock.stay_id.unique().tolist()
list(set(past_shock_lst).difference(set(sepsis3_cut.stay_id.unique())))

[]

In [16]:
len(septic_shock_lst), len(past_shock_lst)

(1353, 1029)

In [21]:
len(list(set(septic_shock_lst).intersection(set(past_shock_lst))))

962